In [1]:
import os
import json
import psycopg2
import datetime
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

# Define the Gmail API scope and credentials file
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
CREDENTIALS_FILE = 'credentials.json'

# Define the PostgreSQL database connection details
DB_HOST = 'localhost'
DB_PORT = '5432'
DB_NAME = 'email_rules'
DB_USER = 'username'
DB_PASSWORD = 'password'

# Define the rules JSON file
RULES_FILE = 'rules.json'

def authenticate_gmail():
    # Load credentials from file or initiate the OAuth flow
    if os.path.exists('token.json'):
        credentials = Credentials.from_authorized_user_file('token.json', SCOPES)
    else:
        flow = InstalledAppFlow.from_client_secrets_file(CREDENTIALS_FILE, SCOPES)
        credentials = flow.run_local_server(port=0)
        with open('token.json', 'w') as token:
            token.write(credentials.to_json())

    # Create the Gmail service using the authenticated credentials
    service = build('gmail', 'v1', credentials=credentials)
    return service

def fetch_emails(service):
    # Fetch a list of emails from the Inbox
    results = service.users().messages().list(userId='me', labelIds=['INBOX']).execute()
    emails = results.get('messages', [])
    return emails

def store_emails(emails):
    # Connect to the PostgreSQL database
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME,
                            user=DB_USER, password=DB_PASSWORD)
    cursor = conn.cursor()

    # Create the emails table if it doesn't exist
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS emails (
            id VARCHAR PRIMARY KEY,
            from_address VARCHAR,
            subject VARCHAR,
            message TEXT,
            received TIMESTAMP,
            unread BOOLEAN
        )
    """)
    conn.commit()

    # Insert or update each email in the database
    for email in emails:
        msg = service.users().messages().get(userId='me', id=email['id']).execute()
        headers = msg['payload']['headers']
        email_data = {
            'id': email['id'],
            'from_address': get_header_value(headers, 'From'),
            'subject': get_header_value(headers, 'Subject'),
            'message': get_message_body(msg),
            'received': datetime.datetime.fromtimestamp(int(msg['internalDate'])/1000.0),
            'unread': 'UNREAD' in email['labelIds']
        }
        cursor.execute("""
            INSERT INTO emails (id, from_address, subject, message, received, unread)
            VALUES (%(id)s, %(from_address)s, %(subject)s, %(message)s, %(received)s, %(unread)s)
            ON CONFLICT (id) DO UPDATE SET
            from_address = %(from_address)s,
            subject = %(subject)s,
            message = %(message)s,
            received = %(received)s,
            unread = %(unread)s
        """, email_data)
        conn.commit()

    # Close the database connection
    cursor.close()
    conn.close()

def get_header_value(headers, name):
    # Get the value of a header field from the list of headers
    for header in headers:
        if header['name'] == name:
            return header['value']
    return ''

def get_message_body(msg):
    # Get the message body from the email payload
    if 'parts' in msg['payload']:
        parts = msg['payload']['parts']
        for part in parts:
            if part['mimeType'] == 'text/plain':
                data = part['body'].get('data')
                if data:
                    return base64.urlsafe_b64decode(data).decode('utf-8')
    return ''

def process_emails(service):
    # Load the rules from the JSON file
    with open(RULES_FILE) as file:
        rules = json.load(file)

    # Connect to the PostgreSQL database
    conn = psycopg2.connect(host=DB_HOST, port=DB_PORT, database=DB_NAME,
                            user=DB_USER, password=DB_PASSWORD)
    cursor = conn.cursor()

    # Fetch all emails from the database
    cursor.execute("SELECT * FROM emails")
    emails = cursor.fetchall()

    # Apply rules and take actions on emails
    for email in emails:
        for rule in rules:
            if rule['predicate'] == 'All':
                if all(match_condition(email, condition) for condition in rule['conditions']):
                    perform_actions(service, email, rule['actions'])
            elif rule['predicate'] == 'Any':
                if any(match_condition(email, condition) for condition in rule['conditions']):
                    perform_actions(service, email, rule['actions'])

    # Close the database connection
    cursor.close()
    conn.close()

def match_condition(email, condition):
    field = condition['field']
    predicate = condition['predicate']
    value = condition['value']

    if field == 'From':
        return match_string_predicate(email['from_address'], predicate, value)
    elif field == 'Subject':
        return match_string_predicate(email['subject'], predicate, value)
    elif field == 'Message':
        return match_string_predicate(email['message'], predicate, value)
    elif field == 'Received':
        return match_date_predicate(email['received'], predicate, value)

def match_string_predicate(text, predicate, value):
    if predicate == 'contains':
        return value.lower() in text.lower()
    elif predicate == 'does_not_contain':
        return value.lower() not in text.lower()
    elif predicate == 'equals':
        return value.lower() == text.lower()
    elif predicate == 'does_not_equal':
        return value.lower() != text.lower()

def match_date_predicate(date, predicate, value):
    if predicate == 'less_than':
        return date < datetime.datetime.now() - datetime.timedelta(days=int(value))
    elif predicate == 'greater_than':
        return date > datetime.datetime.now() - datetime.timedelta(days=int(value))

def perform_actions(service, email, actions):
    for action in actions:
        if action == 'mark_as_read':
            mark_email_as_read(service, email['id'])
        elif action == 'mark_as_unread':
            mark_email_as_unread(service, email['id'])
        elif action.startswith('move_to_'):
            label = action.split('move_to_')[1]
            move_email_to_label(service, email['id'], label)

def mark_email_as_read(service, email_id):
    service.users().messages().modify(userId='me', id=email_id,
                                      body={'removeLabelIds': ['UNREAD']}).execute()

def mark_email_as_unread(service, email_id):
    service.users().messages().modify(userId='me', id=email_id,
                                      body={'addLabelIds': ['UNREAD']}).execute()

def move_email_to_label(service, email_id, label):
    label_id = get_label_id(service, label)
    if label_id:
        service.users().messages().modify(userId='me', id=email_id,
                                          body={'addLabelIds': [label_id]}).execute()

def get_label_id(service, label_name):
    labels = service.users().labels().list(userId='me').execute().get('labels', [])
    for label in labels:
        if label['name'] == label_name:
            return label['id']
    return ''

if __name__ == '__main__':
    # Authenticate to Gmail and fetch emails
    service = authenticate_gmail()
    emails = fetch_emails(service)

    # Store emails in the database
    store_emails(emails)

    # Process emails based on rules and take actions
    process_emails(service)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [2]:
pip install psycopg2

     ---------------------------------------- 1.2/1.2 MB 4.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install google-api-python-client

     ---------------------------------------- 11.3/11.3 MB 5.5 MB/s eta 0:00:00
     ---------------------------------------- 96.9/96.9 kB 2.8 MB/s eta 0:00:00
     -------------------------------------- 120.3/120.3 kB 3.4 MB/s eta 0:00:00
     -------------------------------------- 178.9/178.9 kB 3.6 MB/s eta 0:00:00
     -------------------------------------- 223.6/223.6 kB 4.5 MB/s eta 0:00:00
  Using cached rsa-4.9-py3-none-any.whl (34 kB)
Note: you may need to restart the kernel to use updated packages.


In [10]:
from google_auth_oauthlib.flow import InstalledAppFlow

ModuleNotFoundError: No module named 'google_auth_oauthlib'